# *Importing the Required Libraries*

In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# *Reading the Data from CSV file*

In [2]:
movies_data = pd.read_csv('/content/movies_data.csv')

In [5]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [6]:
movies_data.shape

(4803, 24)

In [7]:
movies_data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

# *Selecting the desired features for the movie recommandation system*

In [8]:
selected_features = ['genres','keywords','original_title','overview','tagline','cast','director']

# *Filling all the missing values with the null string*

In [9]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

# *Combining all the features*

In [10]:
combined_feature = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['original_title']+' '+movies_data['overview']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [11]:
combined_feature

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  Newlyweds A newlywed couple's ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      Shanghai Calling When ambitious New York att...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

# *Converting the text data into feature vectors*

In [13]:
vectorizer = TfidfVectorizer()

In [15]:
feature_vector = vectorizer.fit_transform(combined_feature)

In [17]:
print(feature_vector)

  (0, 4426)	0.13004959062649668
  (0, 14613)	0.08285434427286248
  (0, 23831)	0.14751739050825702
  (0, 18308)	0.1209228707427199
  (0, 16039)	0.16866690346528304
  (0, 26651)	0.11408262788981283
  (0, 30530)	0.14827639152021851
  (0, 25547)	0.1564219964497782
  (0, 24283)	0.16537327333542137
  (0, 31484)	0.14827639152021851
  (0, 31077)	0.17956496748351683
  (0, 24326)	0.10873330498132479
  (0, 19995)	0.03086073726102991
  (0, 31052)	0.0710720050721456
  (0, 9335)	0.1404215434342093
  (0, 5419)	0.162468085668314
  (0, 1041)	0.12041947395189906
  (0, 1278)	0.05170708949059431
  (0, 22177)	0.16387793147534246
  (0, 1318)	0.03128010560257728
  (0, 20243)	0.1586658687241279
  (0, 10681)	0.13630105644048626
  (0, 3074)	0.09806903974305067
  (0, 28357)	0.14277242492893585
  (0, 2776)	0.09641431696741483
  :	:
  (4802, 19815)	0.05838586925209494
  (4802, 12303)	0.07491587425479584
  (4802, 30597)	0.07401988990893983
  (4802, 9624)	0.06749774524446993
  (4802, 20070)	0.0548813572618499
  (480

# *Using Cosine Similarity*

In [19]:
# finding the cosine similarity

similarity = cosine_similarity(feature_vector)

In [20]:
similarity

array([[1.        , 0.0540225 , 0.03084845, ..., 0.02436576, 0.02810208,
        0.00666179],
       [0.0540225 , 1.        , 0.04099378, ..., 0.05361211, 0.03669106,
        0.01553442],
       [0.03084845, 0.04099378, 1.        , ..., 0.0227109 , 0.0425595 ,
        0.01229387],
       ...,
       [0.02436576, 0.05361211, 0.0227109 , ..., 1.        , 0.02988327,
        0.03921517],
       [0.02810208, 0.03669106, 0.0425595 , ..., 0.02988327, 1.        ,
        0.02879873],
       [0.00666179, 0.01553442, 0.01229387, ..., 0.03921517, 0.02879873,
        1.        ]])

In [21]:
similarity.shape

(4803, 4803)

In [22]:
similarity[0]

array([1.        , 0.0540225 , 0.03084845, ..., 0.02436576, 0.02810208,
       0.00666179])

# *Taking name of the movie from the user*

In [63]:
movie_name_user = input("Enter the name of your favourite movie : ")

Enter the name of your favourite movie : sita ramam


# *Finding the clossest match of the user's entered movie*

In [37]:
movie_list = movies_data['original_title'].tolist()

In [38]:
movie_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

# *Finding the close match*

In [64]:
find_close_match = difflib.get_close_matches(movie_name_user,movie_list)

In [65]:
find_close_match

[]

In [45]:
close_match = find_close_match[0]

In [46]:
close_match

'The Lincoln Lawyer'

# *Getting the index value of the closed match movie*

In [47]:
index_of_movie = movies_data[movies_data['original_title']==close_match]['index'].values[0]

In [48]:
index_of_movie

1194

# *Comparing the similarity score of all the movies of the dataset with the user entered movie*

In [49]:
similarity_score = list(enumerate(similarity[index_of_movie]))

In [52]:
similarity_score

[(0, 0.01063544369385046),
 (1, 0.03643926230456826),
 (2, 0.028555228436544924),
 (3, 0.03001524030671527),
 (4, 0.020580566558308973),
 (5, 0.021256989773793766),
 (6, 0.011418840253707133),
 (7, 0.020061413221804134),
 (8, 0.010308735532212381),
 (9, 0.015495065867731098),
 (10, 0.016021990914445464),
 (11, 0.031690029472636924),
 (12, 0.0187697849246521),
 (13, 0.02523036052296841),
 (14, 0.03063891251758793),
 (15, 0.0275530807649401),
 (16, 0.025274926737702975),
 (17, 0.021367269450467262),
 (18, 0.0579370814999654),
 (19, 0.029296605501225086),
 (20, 0.032249225795683054),
 (21, 0.033467093610655536),
 (22, 0.021569495469518314),
 (23, 0.010303309158603235),
 (24, 0.039395156634619476),
 (25, 0.03158786076983142),
 (26, 0.014884900992008385),
 (27, 0.03167506874429553),
 (28, 0.008407659271429193),
 (29, 0.018540002838720558),
 (30, 0.024387400952580362),
 (31, 0.00968463007853929),
 (32, 0.016617772068992752),
 (33, 0.012337736331881139),
 (34, 0.012539094704338782),
 (35, 0.0

In [53]:
len(similarity_score)

4803

In [54]:
sorted_similar_movies = sorted(similarity_score,reverse=True,key=lambda x: x[1])

In [55]:
sorted_similar_movies

[(1194, 1.0),
 (3621, 0.195719983699982),
 (954, 0.1846174638583727),
 (271, 0.1757811439636179),
 (729, 0.15899383239683026),
 (1202, 0.15109058160997751),
 (1915, 0.14377186610143286),
 (1781, 0.13676151142305285),
 (4602, 0.1333086232582939),
 (3634, 0.12596656302980802),
 (1467, 0.11688285178034036),
 (2510, 0.11545437074330513),
 (2533, 0.111994540199492),
 (624, 0.1106514530754851),
 (2363, 0.11001326768384469),
 (1047, 0.10975878417368563),
 (2692, 0.10888210041938),
 (756, 0.10627150635654267),
 (2237, 0.10533264804976052),
 (633, 0.1052137099085715),
 (2090, 0.10264366941196046),
 (415, 0.09991265536308479),
 (3932, 0.09948026544350931),
 (2061, 0.09887514694580606),
 (1156, 0.09849560098333233),
 (1952, 0.0970220288449374),
 (2919, 0.09617047613974218),
 (167, 0.09164113922307257),
 (1647, 0.08876197524780247),
 (4618, 0.0886536155262353),
 (4127, 0.08830857296517533),
 (3098, 0.08703883705467189),
 (952, 0.08593397504161561),
 (2095, 0.0839935159070291),
 (2976, 0.0839443869

# *Suggesting the movies for the user*

In [60]:
print("The suggested movie for you are listed below \n")

i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  movie_title = movies_data[movies_data.index==index]['original_title'].values[0]
  if i<=30:
    print(f"{i} -- {movie_title}")
    i+=1

The suggested movie for you are listed below 

1 -- The Lincoln Lawyer
2 -- Slums of Beverly Hills
3 -- The Judge
4 -- The Island
5 -- A Civil Action
6 -- Legal Eagles
7 -- The Conspirator
8 -- A Walk Among the Tombstones
9 -- 12 Angry Men
10 -- Zero Effect
11 -- The Age of Innocence
12 -- Beverly Hills Cop
13 -- The Woman in Black
14 -- Abraham Lincoln: Vampire Hunter
15 -- Troop Beverly Hills
16 -- The Client
17 -- Thinner
18 -- Intolerable Cruelty
19 -- Alone in the Dark
20 -- Lincoln
21 -- Bringing Down the House
22 -- Dredd
23 -- Thirteen Conversations About One Thing
24 -- Michael Clayton
25 -- A Time to Kill
26 -- The Counselor
27 -- My Cousin Vinny
28 -- Sahara
29 -- Wicker Park
30 -- Nothing But a Man


# **Movie Recommandation System**

In [67]:
def recommand_movie(movies_data,movie_list,similarity,movie_name_user):
  find_close_match = difflib.get_close_matches(movie_name_user,movie_list)
  if len(find_close_match)>0:
    close_match = find_close_match[0]
    index_of_movie = movies_data[movies_data['original_title']==close_match]['index'].values[0]
    similarity_score = list(enumerate(similarity[index_of_movie]))
    sorted_similar_movies = sorted(similarity_score,reverse=True,key=lambda x: x[1])
    print("The suggested movie for you are listed below \n")
    i = 1
    for movie in sorted_similar_movies:
      index = movie[0]
      movie_title = movies_data[movies_data.index==index]['original_title'].values[0]
      if i<=30:
        print(f"{i} -- {movie_title}")
        i+=1
  else:
    print("Sorry not possible to recommand any of the movie\n")
    print("You can try again with another movie\n")

In [69]:
movie_name_user = input("Enter the name of your favourite movie : ")
recommand_movie(movies_data,movie_list,similarity,movie_name_user)

Enter the name of your favourite movie : avatar
The suggested movie for you are listed below 

1 -- Avatar
2 -- Alien
3 -- Lifeforce
4 -- Moonraker
5 -- Aliens
6 -- Lost in Space
7 -- Star Trek Beyond
8 -- Gattaca
9 -- Guardians of the Galaxy
10 -- Zathura: A Space Adventure
11 -- Gravity
12 -- Space Chimps
13 -- Apollo 18
14 -- Lockout
15 -- The Book of Life
16 -- Cargo
17 -- Treasure Planet
18 -- Star Trek Into Darkness
19 -- Sunshine
20 -- Men in Black
21 -- Men in Black II
22 -- Transformers: Dark of the Moon
23 -- Tears of the Sun
24 -- Starship Troopers
25 -- The Right Stuff
26 -- Lara Croft Tomb Raider: The Cradle of Life
27 -- Deep Impact
28 -- Sphere
29 -- Jason X
30 -- Space Cowboys
